In [2]:
!pip install transformers
!pip install datasets
!pip install torch

In [3]:
from datasets import load_dataset

datasets = load_dataset("cifar10")

labels = datasets["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label


In [ ]:
import time
import torch
from transformers import CLIPProcessor, CLIPModel

model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

In [6]:
def homemade_batch(num_img, batch_size=10, start_img=0):
    homemade = []
    num_batches = num_img // batch_size
    extra = num_img % batch_size

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    t_0 = time.perf_counter()

    for i in range(num_batches):
        t1 = time.perf_counter()

        subset = datasets["train"].select(range((i * batch_size + start_img), (i + 1) * batch_size + start_img))
        input = processor(text=labels, images=subset["img"], return_tensors="pt", padding=False).to(device)
        output = model(**input)

        logits = output.logits_per_image
        probs = logits.softmax(dim=1)

        max_prob, max_index = probs.max(dim=1)
        homemade.append([max_prob.cpu().detach(), max_index.cpu().detach()])
        torch.cuda.empty_cache()

        t2 = time.perf_counter()
        print(f"Finished batch {i + 1} of {num_batches} in {t2 - t1} seconds")

    t_3 = time.perf_counter()
    print(f"Finished entire dataset in {t_3 - t_0} seconds")

    return homemade



In [13]:
def prediction_reformat(subset):
    predicted = []
    count = 1
    for i in range(len(result)):
        for k in range(len(subset[0][0])):
            prob = subset[i][0][k].item()
            id = subset[i][1][k].item()
            label = id2label[id]
            predicted.append([i, label, prob, id])

            count += 1

    return predicted

In [21]:
def accuracy(result):
    correct = 0
    total = 0

    all_labels = {}
    for label in datasets["train"].features["label"].names:
        all_labels[label] = 0

    incorrect = {}
    for label in datasets["train"].features["label"].names:
        incorrect[label] = 0


    for i in range(len(result)):
        all_labels[result[i][1]] += 1
        if datasets["train"][i]["label"] == result[i][3]:
            correct += 1
            total += 1
            if total % 50 == 0:
              print(f"Model accurately predicted {result[i][1]} with {result[i][2] * 100}% confidence.")
        else:
            total += 1
            print(f"Model inaccurately predicted {result[i][1]} with {result[i][2] * 100}% confidence.")
            incorrect[result[i][1]] += 1


    print(f"Acurracy: {(correct/total) * 100}%")

    worst_accuracy = []
    for label in all_labels:
        correct =  all_labels[label] - incorrect[label]
        total = all_labels[label]
        print(f"For {label}: Predicted {correct} out of {total} correct. {(correct) / total * 100}% Accuracy")
        worst_accuracy.append([label, correct/total])

    worst_group = min(worst_accuracy, key=lambda x: x[1])
    print(f"The worst performing group is '{worst_group[0]}' with an accuracy of {worst_group[1] * 100}%")


In [22]:
def data_analysis(predictions):
  cleaned = prediction_reformat(predictions)
  final_results = accuracy(cleaned)

In [24]:
result = homemade_batch(5000)
print(result)

Finished batch 1 of 500 in 0.596545416999902 seconds
Finished batch 2 of 500 in 0.750236530000052 seconds
Finished batch 3 of 500 in 0.7110337660000141 seconds
Finished batch 4 of 500 in 0.7189878219999173 seconds
Finished batch 5 of 500 in 0.7182116180000548 seconds
Finished batch 6 of 500 in 0.7201500300000134 seconds
Finished batch 7 of 500 in 0.7219336440000461 seconds
Finished batch 8 of 500 in 0.7249805410000363 seconds
Finished batch 9 of 500 in 0.7266660489999595 seconds
Finished batch 10 of 500 in 0.751069083999937 seconds
Finished batch 11 of 500 in 0.7515203770000198 seconds
Finished batch 12 of 500 in 0.7534011359999795 seconds
Finished batch 13 of 500 in 0.7580375059999369 seconds
Finished batch 14 of 500 in 0.7328281220000008 seconds
Finished batch 15 of 500 in 0.7328600989999359 seconds
Finished batch 16 of 500 in 0.7362740460000623 seconds
Finished batch 17 of 500 in 0.7411615649999703 seconds
Finished batch 18 of 500 in 0.7326110519999247 seconds
Finished batch 19 of 5

In [25]:
data_analysis(result)

Model inaccurately predicted automobile with 30.468297004699707% confidence.
Model inaccurately predicted horse with 26.002177596092224% confidence.
Model inaccurately predicted truck with 47.97724783420563% confidence.
Model inaccurately predicted cat with 55.37012219429016% confidence.
Model inaccurately predicted bird with 46.18670046329498% confidence.
Model inaccurately predicted truck with 36.99069321155548% confidence.
Model inaccurately predicted deer with 69.54557299613953% confidence.
Model inaccurately predicted bird with 68.53422522544861% confidence.
Model accurately predicted dog with 80.16961216926575% confidence.
Model inaccurately predicted deer with 63.88816237449646% confidence.
Model inaccurately predicted dog with 86.51031255722046% confidence.
Model inaccurately predicted ship with 31.448206305503845% confidence.
Model inaccurately predicted truck with 69.19840574264526% confidence.
Model inaccurately predicted cat with 88.7103259563446% confidence.
Model accurate